In [1]:
# https://sedona.apache.org/1.4.1/tutorial/sql/
# https://sedona.apache.org/1.4.1/tutorial/sql/#set-up-dependencies

from sedona.spark import SedonaContext

config = (
    SedonaContext.builder()
    .config(
        "spark.jars.packages",
        "org.apache.sedona:sedona-spark-shaded-3.4_2.12:1.4.1,"
        "org.datasyslab:geotools-wrapper:1.4.0-28.2",
    )
    .getOrCreate()
)

sedona = SedonaContext.create(config)

23/07/28 20:33:04 WARN Utils: Your hostname, Karns-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.40 instead (on interface en0)
23/07/28 20:33:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/kahnwong/.ivy2/cache
The jars for the packages stored in: /Users/kahnwong/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.4_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a427e20c-50d0-4d05-a3fe-7c5e0e63c4fa;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/kahnwong/.local/share/virtualenvs/geoparquet-spark-playground-fuyZVfg8/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-shaded-3.4_2.12;1.4.1 in central
	found org.datasyslab#geotools-wrapper;1.4.0-28.2 in central
:: resolution report :: resolve 89ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.4_2.12;1.4.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.4.0-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-a427e20c-50d0-4d05-a3fe-7c5e0e63c4fa
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/2ms)
23/07/28 20:33:05 WARN NativeCodeLoader: Unable to load

In [2]:
!mkdir -p data
!wget -P data/ https://github.com/opengeospatial/geoparquet/raw/main/examples/example.parquet

--2023-07-28 20:33:09--  https://github.com/opengeospatial/geoparquet/raw/main/examples/example.parquet
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opengeospatial/geoparquet/main/examples/example.parquet [following]
--2023-07-28 20:33:10--  https://raw.githubusercontent.com/opengeospatial/geoparquet/main/examples/example.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27814 (27K) [application/octet-stream]
Saving to: ‘data/example.parquet.1’

example.parquet.1   100%[===================>]  27.16K  --.-KB/s    in 0.001s  

2023-07-28 20:33:10 (29.3 MB/s) - ‘data/example.parquet.1’

In [3]:
df = sedona.read.format("geoparquet").load("data/example.parquet")
# df = spark.read.parquet('data/example.parquet')
df.printSchema()

root
 |-- pop_est: double (nullable = true)
 |-- continent: string (nullable = true)
 |-- name: string (nullable = true)
 |-- iso_a3: string (nullable = true)
 |-- gdp_md_est: long (nullable = true)
 |-- geometry: geometry (nullable = true)



In [4]:
df.show()

+------------+-------------+--------------------+------+----------+--------------------+
|     pop_est|    continent|                name|iso_a3|gdp_md_est|            geometry|
+------------+-------------+--------------------+------+----------+--------------------+
|    889953.0|      Oceania|                Fiji|   FJI|      5496|MULTIPOLYGON (((1...|
| 5.8005463E7|       Africa|            Tanzania|   TZA|     63177|POLYGON ((33.9037...|
|    603253.0|       Africa|           W. Sahara|   ESH|       907|POLYGON ((-8.6655...|
| 3.7589262E7|North America|              Canada|   CAN|   1736425|MULTIPOLYGON (((-...|
|3.28239523E8|North America|United States of ...|   USA|  21433226|MULTIPOLYGON (((-...|
+------------+-------------+--------------------+------+----------+--------------------+



In [5]:
df.createOrReplaceTempView("df")

In [6]:
sedona.sql(
    """
SELECT *, ST_GeoHash(geometry, 5) as geohash
FROM df
ORDER BY geohash
    """
).show()

+------------+-------------+--------------------+------+----------+--------------------+-------+
|     pop_est|    continent|                name|iso_a3|gdp_md_est|            geometry|geohash|
+------------+-------------+--------------------+------+----------+--------------------+-------+
|3.28239523E8|North America|United States of ...|   USA|  21433226|MULTIPOLYGON (((-...|  c250g|
| 3.7589262E7|North America|              Canada|   CAN|   1736425|MULTIPOLYGON (((-...|  cg55k|
|    603253.0|       Africa|           W. Sahara|   ESH|       907|POLYGON ((-8.6655...|  esqf7|
|    889953.0|      Oceania|                Fiji|   FJI|      5496|MULTIPOLYGON (((1...|  khbn2|
| 5.8005463E7|       Africa|            Tanzania|   TZA|     63177|POLYGON ((33.9037...|  kybgb|
+------------+-------------+--------------------+------+----------+--------------------+-------+

